<a href="https://colab.research.google.com/github/gregoryozaki/data-analysis-challenge-on-brazilian-e-commerce/blob/main/dataAnalysisChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ETL**

### **1. Extração dos Dados**

In [2]:
import os
import pandas as pd
import numpy as np
import altair as alt
from google.colab import drive

drive.mount('/content/drive')


data_path = '/content/drive/MyDrive/Brazilian E-Commerce Public Dataset by Olist/'
output_path = os.path.join(data_path, 'processed_merged_df.csv')

print("\nVerificando arquivos...")
required_files = [
    'olist_customers_dataset.csv', 'olist_geolocation_dataset.csv',
    'olist_order_items_dataset.csv', 'olist_order_payments_dataset.csv',
    'olist_order_reviews_dataset.csv', 'olist_orders_dataset.csv',
    'olist_products_dataset.csv', 'olist_sellers_dataset.csv',
    'product_category_name_translation.csv'
]

for file in required_files:
    assert os.path.exists(os.path.join(data_path, file)), f"Arquivo não encontrado: {file}"
print("Todos os arquivos foram encontrados com sucesso.")

def remove_duplicates(df, name):
    dups = df.duplicated().sum()
    if dups > 0:
        df.drop_duplicates(inplace=True)
        print(f"{name}: {dups} duplicatas removidas.")
    else:
        print(f"{name}: sem duplicatas.")
    return df

def convert_to_datetime(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"{col} convertido para datetime.")
    return df

print("\nLendo datasets...")
customers_df = pd.read_csv(os.path.join(data_path, 'olist_customers_dataset.csv'))
geolocation_df = pd.read_csv(os.path.join(data_path, 'olist_geolocation_dataset.csv'))
order_items_df = pd.read_csv(os.path.join(data_path, 'olist_order_items_dataset.csv'))
order_payments_df = pd.read_csv(os.path.join(data_path, 'olist_order_payments_dataset.csv'))
order_reviews_df = pd.read_csv(os.path.join(data_path, 'olist_order_reviews_dataset.csv'))
orders_df = pd.read_csv(os.path.join(data_path, 'olist_orders_dataset.csv'))
products_df = pd.read_csv(os.path.join(data_path, 'olist_products_dataset.csv'))
sellers_df = pd.read_csv(os.path.join(data_path, 'olist_sellers_dataset.csv'))
product_category_translation_df = pd.read_csv(os.path.join(data_path, 'product_category_name_translation.csv'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Verificando arquivos...
Todos os arquivos foram encontrados com sucesso.

Lendo datasets...


### 2. Transformação dos dados

In [3]:
print("\nRemovendo duplicatas...")
customers_df = remove_duplicates(customers_df, "Customers")
geolocation_df = remove_duplicates(geolocation_df, "Geolocation")
order_items_df = remove_duplicates(order_items_df, "Order Items")
order_payments_df = remove_duplicates(order_payments_df, "Order Payments")
order_reviews_df = remove_duplicates(order_reviews_df, "Order Reviews")
orders_df = remove_duplicates(orders_df, "Orders")
products_df = remove_duplicates(products_df, "Products")
sellers_df = remove_duplicates(sellers_df, "Sellers")
product_category_translation_df = remove_duplicates(product_category_translation_df, "Category Translation")

print("\nConvertendo colunas de data para datetime...")
orders_df = convert_to_datetime(orders_df, [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
])
order_items_df = convert_to_datetime(order_items_df, ['shipping_limit_date'])
order_reviews_df = convert_to_datetime(order_reviews_df, ['review_creation_date', 'review_answer_timestamp'])

print("\nAgregando dados de geolocalização por CEP...")
geolocation_agg_df = geolocation_df.groupby('geolocation_zip_code_prefix').agg(
    geolocation_lat=('geolocation_lat', 'mean'),
    geolocation_lng=('geolocation_lng', 'mean')
).reset_index()

print("\nFazendo merge dos DataFrames...")
merged_df = pd.merge(orders_df, customers_df, on='customer_id', how='left')
merged_df = pd.merge(merged_df, order_items_df, on='order_id', how='left')
merged_df = pd.merge(merged_df, products_df, on='product_id', how='left')
merged_df = pd.merge(merged_df, product_category_translation_df, on='product_category_name', how='left')
merged_df = pd.merge(merged_df, order_payments_df, on='order_id', how='left')
merged_df = pd.merge(merged_df, order_reviews_df, on='order_id', how='left')
merged_df = pd.merge(merged_df, sellers_df, on='seller_id', how='left')

print("\nAdicionando geolocalização de clientes e vendedores...")

customer_geo_df = geolocation_agg_df.rename(columns={
    'geolocation_zip_code_prefix': 'customer_zip_code_prefix',
    'geolocation_lat': 'customer_geolocation_lat',
    'geolocation_lng': 'customer_geolocation_lng'
})
merged_df = pd.merge(merged_df, customer_geo_df, on='customer_zip_code_prefix', how='left')

seller_geo_df = geolocation_agg_df.rename(columns={
    'geolocation_zip_code_prefix': 'seller_zip_code_prefix',
    'geolocation_lat': 'seller_geolocation_lat',
    'geolocation_lng': 'seller_geolocation_lng'
})
merged_df = pd.merge(merged_df, seller_geo_df, on='seller_zip_code_prefix', how='left')

print("\nTratando valores nulos em categorias de produtos...")
merged_df['product_category_name_english'] = merged_df['product_category_name_english'].fillna(
    merged_df['product_category_name']
).fillna('Unknown')



Removendo duplicatas...
Customers: sem duplicatas.
Geolocation: 261831 duplicatas removidas.
Order Items: sem duplicatas.
Order Payments: sem duplicatas.
Order Reviews: sem duplicatas.
Orders: sem duplicatas.
Products: sem duplicatas.
Sellers: sem duplicatas.
Category Translation: sem duplicatas.

Convertendo colunas de data para datetime...
order_purchase_timestamp convertido para datetime.
order_approved_at convertido para datetime.
order_delivered_carrier_date convertido para datetime.
order_delivered_customer_date convertido para datetime.
order_estimated_delivery_date convertido para datetime.
shipping_limit_date convertido para datetime.
review_creation_date convertido para datetime.
review_answer_timestamp convertido para datetime.

Agregando dados de geolocalização por CEP...

Fazendo merge dos DataFrames...

Adicionando geolocalização de clientes e vendedores...

Tratando valores nulos em categorias de produtos...


### **3. Carregamento dos dados**

In [4]:
print("\nETL concluído com sucesso!")
print("DataFrame final pronto para análise:")
print(merged_df.head())
print(merged_df.info())

merged_df.to_csv(output_path, index=False)
print(f"DataFrame salvo em: {output_path}")


ETL concluído com sucesso!
DataFrame final pronto para análise:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
2  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
3  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
4  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   

  order_status order_purchase_timestamp   order_approved_at  \
0    delivered      2017-10-02 10:56:33 2017-10-02 11:07:15   
1    delivered      2017-10-02 10:56:33 2017-10-02 11:07:15   
2    delivered      2017-10-02 10:56:33 2017-10-02 11:07:15   
3    delivered      2018-07-24 20:41:37 2018-07-26 03:24:27   
4    delivered      2018-08-08 08:38:49 2018-08-08 08:55:23   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 2

# **Análise Exploratória**

## **1. Análise de Performance de Vendas**

**Tendências de Crescimento:** Analisar as tendências de crescimento de vendas ao
longo do tempo para diferentes categorias de produtos

In [17]:
merged_df['order_month'] = merged_df['order_purchase_timestamp'].dt.to_period('M').astype(str)

growth_df = merged_df.groupby(['order_month', 'product_category_name_english']).agg(
    total_sales=('price', 'sum'),
    total_orders=('order_id', 'nunique')
).reset_index()

top_cats = (growth_df.groupby('product_category_name_english')['total_sales']
            .sum().sort_values(ascending=False).head(5).index)

growth_df = growth_df[growth_df['product_category_name_english'].isin(top_cats)]

chart = alt.Chart(growth_df).mark_line(point=True).encode(
    x='order_month:T',
    y='total_sales:Q',
    color='product_category_name_english:N',
    tooltip=['order_month', 'product_category_name_english', 'total_sales']
).properties(
    title='Tendência de Crescimento das Vendas por Categoria (Top 5)'
).interactive()

chart

alt.Chart(...)

## **2. Análise de logística**

**Prazos de Entrega:** Calcular o tempo médio de entrega e identificar os fatores que influenciam atrasos nas entregas.

In [22]:
merged_df['delivery_time'] = (
    merged_df['order_delivered_customer_date'] - merged_df['order_purchase_timestamp']
).dt.days

merged_df['delivery_delay'] = (
    merged_df['order_delivered_customer_date'] - merged_df['order_estimated_delivery_date']
).dt.days

merged_df['is_late'] = merged_df['delivery_delay'] > 0

print("Tempo médio de entrega (em dias):", merged_df['delivery_time'].mean())
print("Mediana:", merged_df['delivery_time'].median())

percent_late = merged_df['is_late'].mean() * 100
print(f"{percent_late:.2f}% dos pedidos foram entregues com atraso.")

delivery_by_state = merged_df.groupby('customer_state')['delivery_time'].mean().sort_values()

delivery_by_category = merged_df.groupby('product_category_name_english')['delivery_time'].mean().sort_values()

chart_state = alt.Chart(delivery_by_state.reset_index()).mark_bar().encode(
    x='delivery_time:Q',
    y=alt.Y('customer_state:N', sort='-x'),
    tooltip=['customer_state', 'delivery_time']
).properties(
    title='Tempo Médio de Entrega por Estado'
)

chart_state
late_by_category = merged_df.groupby('product_category_name_english')['is_late'].mean().sort_values(ascending=False).reset_index()

chart_late = alt.Chart(late_by_category.head(10)).mark_bar().encode(
    x='is_late:Q',
    y=alt.Y('product_category_name_english:N', sort='-x'),
    tooltip=['product_category_name_english', 'is_late']
).properties(
    title='Top 10 Categorias com Mais Atrasos'
)

chart_late


Tempo médio de entrega (em dias): 12.022588617548953
Mediana: 10.0
6.34% dos pedidos foram entregues com atraso.


alt.Chart(...)

## **3. Análise de Satisfação do Cliente**

**Impacto dos Atrasos na Satisfação do Cliente:** Examinar a relação entre atrasos na entrega e a satisfação do cliente.

In [12]:
review_df = merged_df[
    merged_df['review_score'].notnull() &
    merged_df['order_delivered_customer_date'].notnull() &
    merged_df['order_estimated_delivery_date'].notnull()
].copy()

review_df['delivery_delay'] = (
    review_df['order_delivered_customer_date'] - review_df['order_estimated_delivery_date']
).dt.days

review_df['is_late'] = review_df['delivery_delay'] > 0

mean_score_late = review_df[review_df['is_late']]['review_score'].mean()
mean_score_on_time = review_df[~review_df['is_late']]['review_score'].mean()

print(f"Avaliação média com atraso: {mean_score_late:.2f}")
print(f"Avaliação média sem atraso: {mean_score_on_time:.2f}")

score_by_late = review_df.groupby(['is_late', 'review_score']).size().reset_index(name='count')

chart = alt.Chart(score_by_late).mark_bar().encode(
    x=alt.X('review_score:O', title='Nota do Cliente'),
    y='count:Q',
    color='is_late:N',
    column='is_late:N',
    tooltip=['review_score', 'count']
).properties(
    title='Distribuição das Avaliações com e sem Atraso'
)

chart


Avaliação média com atraso: 2.25
Avaliação média sem atraso: 4.21


alt.Chart(...)

## **4. Análise Financeira**


**Análise de Lucratividade por Categoria:** Calcular a lucratividade de diferentes categorias de produtos, levando em conta o custo dos produtos e o preço de venda.

In [13]:
profit_df = merged_df[
    merged_df['price'].notnull() &
    merged_df['freight_value'].notnull() &
    merged_df['product_category_name_english'].notnull()
].copy()

profit_df['estimated_cost'] = profit_df['price'] * 0.40
profit_df['gross_profit'] = profit_df['price'] - profit_df['estimated_cost']
profit_df['net_profit'] = profit_df['gross_profit'] - profit_df['freight_value']

category_profit = profit_df.groupby('product_category_name_english').agg(
    total_revenue=('price', 'sum'),
    total_cost=('estimated_cost', 'sum'),
    total_freight=('freight_value', 'sum'),
    total_net_profit=('net_profit', 'sum'),
    order_count=('order_id', 'nunique')
).reset_index()

category_profit['avg_profit_per_order'] = category_profit['total_net_profit'] / category_profit['order_count']

category_profit = category_profit.sort_values(by='total_net_profit', ascending=False)

chart_profit = alt.Chart(category_profit.head(10)).mark_bar().encode(
    x='total_net_profit:Q',
    y=alt.Y('product_category_name_english:N', sort='-x'),
    tooltip=['product_category_name_english', 'total_net_profit']
).properties(
    title='Top 10 Categorias Mais Lucrativas'
)

chart_profit


alt.Chart(...)

## **5. Análise de Marketing**

**Eficácia de Campanhas Promocionais:** Avaliar o impacto de campanhas
promocionais e descontos no volume de vendas e na aquisição de novos clientes.

In [19]:
merged_df['order_month'] = merged_df['order_purchase_timestamp'].dt.to_period('M')
merged_df['order_month_str'] = merged_df['order_month'].astype(str)

monthly_sales = merged_df.groupby('order_month_str').agg(
    total_sales=('price', 'sum'),
    total_orders=('order_id', 'nunique'),
    new_customers=('customer_unique_id', 'nunique')
).reset_index()

promo_months = ['2017-11', '2017-12', '2018-11', '2018-12']
monthly_sales['is_promotional'] = monthly_sales['order_month_str'].isin(promo_months)

promo_stats = monthly_sales.groupby('is_promotional').agg(
    avg_sales=('total_sales', 'mean'),
    avg_orders=('total_orders', 'mean'),
    avg_new_customers=('new_customers', 'mean')
).reset_index()

print("Comparativo entre meses promocionais e não promocionais:")
print(promo_stats)

import altair as alt

chart = alt.Chart(monthly_sales).mark_bar().encode(
    x=alt.X('order_month_str:N', title='Mês'),
    y=alt.Y('total_sales:Q', title='Vendas Totais'),
    color=alt.Color('is_promotional:N', legend=alt.Legend(title='Promoção')),
    tooltip=['order_month_str', 'total_sales', 'total_orders', 'new_customers']
).properties(
    title='Vendas Mensais com Destaque para Períodos Promocionais',
    width=700,
    height=400
)

chart

Comparativo entre meses promocionais e não promocionais:
   is_promotional      avg_sales   avg_orders  avg_new_customers
0           False  541089.283913  3748.869565        3696.217391
1            True  914323.060000  6608.500000        6516.500000


alt.Chart(...)